In [1]:
import pyspark
from pyspark.sql import SparkSession
app_name = "bubbly"
master = "local[*]"
spark = (SparkSession.builder
    .master(master)
    .config("spark.driver.cores", 1)
    .appName(app_name)
    .getOrCreate() )
sc = spark.sparkContext
print ('SparkContext created')

SparkContext created


In [2]:
# Cargamos el fichero desde hdfs
#accidenteData = spark.read.csv ('hdfs://localhost:9000/user/ubuntu/accidentes/2018.csv',header=True)
#accidenteData = spark.read.csv ('file:///home/ubuntu/Downloads/2018_Accidentalidad.csv',header=True)
accidenteData = spark.read.csv ('file:///home/jose/Descargas/2018.csv',header=True)

AnalysisException: 'Path does not exist: file:/home/jose/Descargas/2018.csv;'

In [3]:
import pyspark.sql.functions as func
from pyspark.sql.types import DoubleType

# Reducimos las columnas que tenemos que utilizar mediante la Select
# y aplicamos las funciones necesarias a los datos
accRed=accidenteData\
    .select(
        func.when (\
                (accidenteData['Nº']=='0'),\
#                pone calle 'esquina' calle
                func.regexp_replace (accidenteData['LUGAR ACCIDENTE'], '() - ', ' ESQUINA ')\
            )\
            .otherwise(
                func.concat (\
                    func.regexp_replace (accidenteData['LUGAR ACCIDENTE'],' NUM\s*$',' '), \
                    accidenteData['Nº'], \
                    func.lit(', MADRID ESPAÑA')\
                )\
            )\
            .alias('direccion') ,\
#        func.when (\
#                (accidenteData['Nº']=='0'),\
#                None\
#            )\
#            .when (\
#                (accidenteData['LUGAR ACCIDENTE'].contains (' KM.')) & \
#                (accidenteData['Nº'].cast(DoubleType()) > 100) ,\
#                accidenteData['Nº'].cast(DoubleType()) / 1000\
#            )\
#            .otherwise(accidenteData['Nº'].cast(DoubleType()))\
#            .alias('numNorm') ,\
        accidenteData['LUGAR ACCIDENTE'].alias('calle'),\
        accidenteData['Nº'].alias('num')\
    )\
    .dropDuplicates ()



NameError: name 'accidenteData' is not defined

### Para geolocalizar las direcciones utilizamos la libreria geocoder de python.
### Se instala con el comando "pip install geocoder"

In [84]:
import geocoder
from pyspark.sql.types import StringType, IntegerType

g=geocoder.osm('calle de dolores barranco num 44, madrid españa')
def getGeo (address):
    geo = geocode.osm (address)
    return geo.x

func_getGeo = func.udf (getGeo,IntegerType())

In [85]:
accGeo = accRed.limit(10).withColumn('coordenadas',func_getGeo(accRed.direccion))
accRed.limi(10).toPandas()

In [86]:
accGeo.show()

Py4JJavaError: An error occurred while calling o1093.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 50.0 failed 1 times, most recent failure: Lost task 0.0 in stage 50.0 (TID 4251, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/jose/spark/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/jose/spark/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/jose/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 342, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/home/jose/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/home/jose/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 331, in _batched
    for item in iterator: